In [1]:
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten
from keras.models import Sequential, Model, load_model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras import backend as K

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
#x_train = x_train / 255.0
#x_test = x_test / 255.0

x_train = preprocess_input(x_train, mode='caffe')
x_test = preprocess_input(x_test, mode='caffe')

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [3]:
base = VGG16(include_top=False, input_shape=(32, 32, 3))
base.trainable = False

x = Flatten(name='flatten')(base.output)
x = Dense(56, activation='relu', name='fc1')(x)
x = Dense(56, activation='relu', name='fc2')(x)
x = Dense(10)(x)
x = Activation('softmax', name='prediction')(x)

model = Model(inputs=base.input, outputs=x)

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [5]:
#print(len(model.trainable_weights))
for layer in model.layers[:20]:
    layer.trainable = False

print([layer.trainable for layer in model.layers])

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True]


In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [7]:
history = model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 22s 449us/step - loss: 1.5705 - acc: 0.5178 - val_loss: 1.1466 - val_acc: 0.6044
Epoch 2/5
50000/50000 [==============================] - 21s 420us/step - loss: 1.0508 - acc: 0.6344 - val_loss: 1.0738 - val_acc: 0.6271
Epoch 3/5
50000/50000 [==============================] - 21s 419us/step - loss: 0.9639 - acc: 0.6654 - val_loss: 1.0210 - val_acc: 0.6515
Epoch 4/5
50000/50000 [==============================] - 21s 412us/step - loss: 0.9088 - acc: 0.6833 - val_loss: 1.0278 - val_acc: 0.6521
Epoch 5/5
50000/50000 [==============================] - 20s 409us/step - loss: 0.8627 - acc: 0.6983 - val_loss: 1.0398 - val_acc: 0.6580


In [8]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 3s 313us/step


[0.18599436459541321, 0.9276999905586243]

In [9]:
model.save('../tmp/cifar10_vgg16_pretrain_test.h5')